In [1]:
import os
import plotly
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from plotly.subplots import make_subplots
from parcllabs.beta.charting.utils import create_labs_logo_dict
from parcllabs.beta.charting.styling import default_style_config as style_config


In [2]:
client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    turbo_mode=True,
    limit=12 # set default limit
)

In [23]:
# in this case, lets look at US market overall
us_market = client.search.markets.retrieve(
    query='United States',
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    limit=1
)

metros = client.search.markets.retrieve(
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    limit=100,
    location_type='CBSA'
)

markets = pd.concat([us_market, metros])
market_ids = markets['parcl_id'].unique().tolist()
markets['state'] = markets['name'].apply(lambda x: x.split(',')[-1].strip().upper().split('-')[0])
markets['clean_name'] = markets.apply(lambda x: f"{x['name'].split('-')[0].split(',')[0].strip()}, {x['state']}", axis=1)
markets['clean_name'] = markets['clean_name'].replace({'United States Of America, UNITED STATES OF AMERICA': 'USA'})

In [4]:
# GET TOTAL STOCK
stock = client.portfolio_metrics.sf_housing_stock_ownership.retrieve(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2024-08-01',
    auto_paginate=True
)

stock = stock[['parcl_id', 'pct_sf_housing_stock_portfolio_1000_plus', 'count_portfolio_1000_plus']]


acq = client.portfolio_metrics.sf_housing_event_counts.retrieve(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2024-06-01',
    auto_paginate=True,
    portfolio_size='PORTFOLIO_1000_PLUS'
)

acq['net'] = acq['acquisitions'] - acq['dispositions']

acq_3 = acq.groupby('parcl_id').agg({'acquisitions': 'sum', 'dispositions': 'sum', 'net': 'sum'}).reset_index()
acq_3 = pd.merge(acq_3, markets[['parcl_id', 'clean_name']], on='parcl_id', how='left')
acq_3.sort_values('net', ascending=False, inplace=True)

data: {'parcl_id': ['5826765', '2900187', '2900078', '2899845', '2899734', '2899967', '2900475', '2900244', '2900128', '2887280', '2899625', '2900245', '2900336', '2900295', '2899753', '2900353', '2900137', '2900332', '2900417', '2899750', '2887292', '2900321', '2900213', '2899841', '2900331', '2900266', '2900315', '2900251', '2887289', '2900049', '2899647', '2900012', '2899671', '2899979', '2899654', '2900174', '2900338', '2900462', '2900275', '2899989', '2900134', '2900205', '2900282', '2900122', '2900292', '2900079', '2900182', '2900329', '2899944', '2899645', '2899611', '2899916', '2900301', '2900436', '2900438', '2900447', '2899715', '2899896', '2900208', '2899636', '2899929', '2899862', '2887291', '2899858', '2900030', '2900116', '2899589', '2899787', '2900181', '2899867', '2900192', '2900223', '2899666', '2899742', '2899840', '2900404', '2899924', '2899822', '2899621', '2899664', '2900070', '2900041', '2899752', '2899854', '2900271', '2900201', '2900391', '2900089', '2900503', '

In [5]:
acq_3.tail(20)

,parcl_id,acquisitions,dispositions,net,clean_name
42,2899989,300,322,-22,"Jacksonville, FL"
66,2900229,52,75,-23,"Palm Bay, FL"
20,2899750,114,138,-24,"Denver, CO"
17,2899734,643,667,-24,"Dallas, TX"
40,2899979,335,363,-28,"Indianapolis, IN"
48,2900078,54,84,-30,"Los Angeles, CA"
15,2899671,66,97,-31,"Columbus, OH"
60,2900192,102,135,-33,"North Port, FL"
46,2900049,399,436,-37,"Las Vegas, NV"
76,2900295,86,124,-38,"Riverside, CA"


In [6]:
# find starting value of portfolio sizes for each market
start_stock = client.portfolio_metrics.sf_housing_stock_ownership.retrieve(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2024-06-01',
    end_date='2024-06-01',
    auto_paginate=True
)

start_stock = start_stock[['parcl_id', 'count_portfolio_1000_plus']].rename(columns={'count_portfolio_1000_plus': 'start_portfolio_size'})
stock = stock.rename(columns={
    'count_portfolio_1000_plus': 'current_portfolio_size'
})

data = pd.merge(acq_3, stock, on='parcl_id')
data = pd.merge(data, start_stock, on='parcl_id')
data.head()

data: {'parcl_id': ['5826765', '2900187', '2900078', '2899845', '2899734', '2899967', '2900475', '2900244', '2900128', '2887280', '2899625', '2900245', '2900336', '2900295', '2899753', '2900353', '2900137', '2900332', '2900417', '2899750', '2887292', '2900321', '2900213', '2899841', '2900331', '2900266', '2900315', '2900251', '2887289', '2900049', '2899647', '2900012', '2899671', '2899979', '2899654', '2900174', '2900338', '2900462', '2900275', '2899989', '2900134', '2900205', '2900282', '2900122', '2900292', '2900079', '2900182', '2900329', '2899944', '2899645', '2899611', '2899916', '2900301', '2900436', '2900438', '2900447', '2899715', '2899896', '2900208', '2899636', '2899929', '2899862', '2887291', '2899858', '2900030', '2900116', '2899589', '2899787', '2900181', '2899867', '2900192', '2900223', '2899666', '2899742', '2899840', '2900404', '2899924', '2899822', '2899621', '2899664', '2900070', '2900041', '2899752', '2899854', '2900271', '2900201', '2900391', '2900089', '2900503', '

,parcl_id,acquisitions,dispositions,net,clean_name,pct_sf_housing_stock_portfolio_1000_plus,current_portfolio_size,start_portfolio_size
0,2887280,1026,923,103,"Atlanta, GA",3.80,63200,71807
1,2899845,253,183,70,"Chicago, IL",0.66,13197,10639
2,2899841,443,415,28,"Charlotte, NC",3.40,26364,25188
3,2899621,45,20,25,"Boise City, ID",0.75,1830,1526
4,2899967,268,246,22,"Houston, TX",1.82,32588,23078


In [7]:
data['port_start'] = data['current_portfolio_size'] - data['net']
data['port_size_change'] = (data['current_portfolio_size'] - data['port_start'])/data['port_start']
data

,parcl_id,acquisitions,dispositions,net,clean_name,pct_sf_housing_stock_portfolio_1000_plus,current_portfolio_size,start_portfolio_size,port_start,port_size_change
0,2887280,1026,923,103,"Atlanta, GA",3.80,63200,71807,63097,0.001632
1,2899845,253,183,70,"Chicago, IL",0.66,13197,10639,13127,0.005333
2,2899841,443,415,28,"Charlotte, NC",3.40,26364,25188,26336,0.001063
3,2899621,45,20,25,"Boise City, ID",0.75,1830,1526,1805,0.013850
4,2899967,268,246,22,"Houston, TX",1.82,32588,23078,32566,0.000676
...,...,...,...,...,...,...,...,...,...,...
96,2900137,103,158,-55,"Minneapolis, MN",0.48,4488,3762,4543,-0.012107
97,2899753,404,459,-55,"Detroit, MI",1.03,13075,12545,13130,-0.004189
98,2899664,28,84,-56,"Colorado Springs, CO",1.46,3042,2973,3098,-0.018076
99,2900245,640,746,-106,"Phoenix, AZ",2.45,32084,32579,32190,-0.003293


In [8]:
data.loc[data['parcl_id']==5826765]

,parcl_id,acquisitions,dispositions,net,clean_name,pct_sf_housing_stock_portfolio_1000_plus,current_portfolio_size,start_portfolio_size,port_start,port_size_change
100,5826765,10345,11192,-847,USA,0.74,611388,545680,612235,-0.001383


In [9]:
listings = client.portfolio_metrics.sf_new_listings_for_sale_rolling_counts.retrieve(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2024-04-15',
    auto_paginate=True,
    portfolio_size='PORTFOLIO_1000_PLUS'
)

data: {'parcl_id': ['5826765', '2900187', '2900078', '2899845', '2899734', '2899967', '2900475', '2900244', '2900128', '2887280', '2899625', '2900245', '2900336', '2900295', '2899753', '2900353', '2900137', '2900332', '2900417', '2899750', '2887292', '2900321', '2900213', '2899841', '2900331', '2900266', '2900315', '2900251', '2887289', '2900049', '2899647', '2900012', '2899671', '2899979', '2899654', '2900174', '2900338', '2900462', '2900275', '2899989', '2900134', '2900205', '2900282', '2900122', '2900292', '2900079', '2900182', '2900329', '2899944', '2899645', '2899611', '2899916', '2900301', '2900436', '2900438', '2900447', '2899715', '2899896', '2900208', '2899636', '2899929', '2899862', '2887291', '2899858', '2900030', '2900116', '2899589', '2899787', '2900181', '2899867', '2900192', '2900223', '2899666', '2899742', '2899840', '2900404', '2899924', '2899822', '2899621', '2899664', '2900070', '2900041', '2899752', '2899854', '2900271', '2900201', '2900391', '2900089', '2900503', '

In [10]:
listings = listings.sort_values(['pct_sf_for_sale_market_rolling_30_day']).merge(markets[['parcl_id', 'clean_name']], on='parcl_id', how='left')

In [12]:
listings[['clean_name', 'parcl_id']].drop_duplicates().values

array([['Omaha, NE', 2900208],
       ['Knoxville, TN', 2900030],
       ['Providence, RI', 2900275],
       ['Bridgeport, CT', 2899636],
       ['San Jose, CA', 2900338],
       ['Grand Rapids, MI', 2899916],
       ['Hartford, CT', 2899944],
       ['Des Moines, IA', 2899752],
       ['Spokane, WA', 2900389],
       ['Rochester, NY', 2900301],
       ['New Orleans, LA', 2900182],
       ['Toledo, OH', 2900429],
       ['Allentown, PA', 2899867],
       ['El Paso, TX', 2899787],
       ['Mcallen, TX', 2900116],
       ['Baton Rouge, LA', 2899589],
       ['Provo, UT', 2900276],
       ['Madison, WI', 2900089],
       ['Fresno, CA', 2899715],
       ['Richmond, VA', 2900292],
       ['Buffalo, NY', 2899645],
       ['Little Rock, AR', 2900070],
       ['Boston, MA', 2899625],
       ['Milwaukee, WI', 2900134],
       ['Greenville, SC', 2899929],
       ['Harrisburg, PA', 2899941],
       ['Jackson, MS', 2899985],
       ['Memphis, TN', 2900122],
       ['Louisville/Jefferson County, KY

In [19]:
fl_pids = markets.loc[markets['state']=='FL']['parcl_id'].tolist()
fl = listings.loc[listings['parcl_id'].isin(fl_pids)]

fl['pct_sf_for_sale_market_rolling_7_day'] = fl['pct_sf_for_sale_market_rolling_7_day']/100
fl['pct_sf_for_sale_market_rolling_30_day'] = fl['pct_sf_for_sale_market_rolling_30_day']/100
fl['pct_sf_for_sale_market_rolling_60_day'] = fl['pct_sf_for_sale_market_rolling_60_day']/100


/var/folders/cs/pgg3ckbn3d397n3_x2b312k80000gn/T/ipykernel_54553/1342805152.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/cs/pgg3ckbn3d397n3_x2b312k80000gn/T/ipykernel_54553/1342805152.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/cs/pgg3ckbn3d397n3_x2b312k80000gn/T/ipykernel_54553/1342805152.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [22]:
# Get max date for chart
max_date_for_chart = fl['date'].max().date()
max_date_for_chart = max_date_for_chart.strftime('%B %d, %Y')
fl = fl.sort_values(['parcl_id', 'date'])

WINDOW_PERIOD = 30

CHART_WIDTH = 1600
CHART_HEIGHT = 800
# Create the line chart using Plotly Express
fig = px.line(
    fl,
    x='date',
    y=f'pct_sf_for_sale_market_rolling_{WINDOW_PERIOD}_day',
    color='clean_name',
    line_group='clean_name',
    labels={f'pct_sf_for_sale_market_rolling_{WINDOW_PERIOD}_day': '% of SFH Inventory'},
    title=f"<span style='font-size:20px;'>% of Single Family Home Listings coming from Institutional Portfolios (1000+ Units)</span><br><span style='font-size:12px; font-style: italic;'>{max_date_for_chart} - Rolling {WINDOW_PERIOD} Days</span>"
)

# Define the cities to be highlighted in red
highlighted_cities = []# ['North Port, FL', 'Tampa, FL', 'Lakeland, FL', 'Jacksonville, FL']

# Update traces to apply specific styles
for trace in fig.data:
    if trace.name in highlighted_cities:
        trace.update(
            line=dict(color='red', width=3, dash='dash'),
            opacity=1
        )
    else:
        trace.update(
            line=dict(color='lightblue', dash='dash', width=2),
            opacity=0.8
        )
    # Remove text annotations from traces
    trace.update(
        mode='lines'
    )

# Find the latest date in the dataset
latest_date = max(fl['date'])

# Add annotations for each line on the far right
annotations = []
y_positions = []

for trace in fig.data:
    # Get the last y-value for each clean_name
    last_y_value = fl[
        (fl['clean_name'] == trace.name) &
        (fl['date'] == latest_date)
    ][f'pct_sf_for_sale_market_rolling_{WINDOW_PERIOD}_day'].values[0]
    
    # Only add the annotation if it doesn't overlap with existing annotations
    if not any(abs(last_y_value - y) < 0.002 for y in y_positions):  # Adjust threshold as needed
        annotations.append(dict(
            x=latest_date,
            y=last_y_value,
            xref='x',
            yref='y',
            text=trace.name,
            showarrow=False,
            xanchor='left',
            font=dict(size=12)  # Adjust the font size if needed
        ))
        y_positions.append(last_y_value)

fig.add_layout_image(
    create_labs_logo_dict()
)

# Update layout for axes, title, and other styling
fig.update_layout(
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
    xaxis=dict(
        title='',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        # tickangle=style_config['tick_angle'],
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    yaxis=dict(
        title='% of Single Family Home Listings',
        showgrid=style_config['showgrid'],
        gridwidth=style_config['gridwidth'],
        gridcolor=style_config['grid_color'],
        tickfont=style_config['axis_font'],
        zeroline=False,
        tickformat='.0%',
        linecolor=style_config['line_color_axis'],
        linewidth=style_config['linewidth'],
        titlefont=style_config['title_font_axis']
    ),
    plot_bgcolor=style_config['background_color'],
    paper_bgcolor=style_config['background_color'],
    font=dict(color=style_config['font_color']),
    showlegend=False,  # Remove the legend
    margin=dict(l=40, r=40, t=100, b=40),
    title={
        'y': 0.96,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    annotations=annotations  # Add annotations
)

fig.show()


In [168]:
out = fl[['parcl_id', 'clean_name', 'portfolio_size', 'date', 'pct_sf_for_sale_market_rolling_60_day']].reset_index(drop=True)
out.to_csv('fl_institutional_listings.csv', index=False)

In [169]:
out.head()

,parcl_id,clean_name,portfolio_size,date,pct_sf_for_sale_market_rolling_60_day
0,2899748,"Deltona, FL",PORTFOLIO_1000_PLUS,2024-04-15,0.0109
1,2900192,"North Port, FL",PORTFOLIO_1000_PLUS,2024-04-15,0.0163
2,2900128,"Miami, FL",PORTFOLIO_1000_PLUS,2024-04-15,0.0107
3,2900041,"Lakeland, FL",PORTFOLIO_1000_PLUS,2024-04-15,0.0240
4,2900417,"Tampa, FL",PORTFOLIO_1000_PLUS,2024-04-15,0.0212


In [114]:
data_fl = data.loc[data['parcl_id'].isin(fl_pids)].sort_values('net', ascending=False)

In [113]:
import os
from itertools import islice

import plotly
import plotly.io as pio
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from plotly.subplots import make_subplots
from parcllabs.beta.charting.utils import create_labs_logo_dict
from parcllabs.beta.charting.styling import default_style_config as style_config
import plotly.graph_objects as go


import plotly.io as pio

### Global Settings
ADD_WATERMARK = False
ADD_LOGO = True
BG_COLOR = '#080D16'
FIRST_COLOR = '#448CF2'
SECOND_COLOR = '#3ED8ED'
THIRD_COLOR = '#8CE9A6'
FOURTH_COLOR = '#9ED84C'
TITLE_Y = 0.97
TITLE_X = 0.5
TITLE_FONT_SIZE = 20
TITLE_Y_AXIS_FONT_SIZE = 10
CHART_BORDER_COLOR = '#132D59'
GRID_LINES_COLOR = '#0A1B39'
Y_AXIS_TITLE_STANDOFF = 8


# Create a custom template with Inter font
pio.templates["custom"] = pio.templates["plotly"].update(
    layout=dict(
        font=dict(family="Inter, Arial, sans-serif")
    )
)

# Set the custom template as default
pio.templates.default = "custom"

def add_watermark(fig):
    # Add annotations
    if ADD_WATERMARK:
        fig.add_annotation(
            text="@ParclLabs",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=60, color="rgba(255,255,255,0.1)"),
            textangle=-30
        )
    return fig

def add_logo(fig, x=1, sizex=0.1, sizey=0.1):
    if ADD_LOGO:
        fig.add_layout_image(
            create_labs_logo_dict(color='white', x=x, sizex=sizex, sizey=sizey),
        )
    return fig

def lab_plot_utils(fig, logo_x=1, logo_sizex=0.1, logo_sizey=0.1):
    fig = add_watermark(fig)
    fig = add_logo(fig, x=logo_x, sizex=logo_sizex, sizey=logo_sizey)
    return fig

def update_x_axis_tick_marks(fig, tickformat=None, tickangle=45):
    # Update x-axis
    fig.update_xaxes(
        title_text="",
        showgrid=True,
        gridwidth=0.5,
        gridcolor=GRID_LINES_COLOR,
        tickangle=tickangle,
        dtick="M6",
        tickformat=tickformat,
        tickfont=dict(size=12),
        linecolor=CHART_BORDER_COLOR,
        linewidth=2,
        mirror=True,
        title_font=dict(family="Inter")
    )

    return fig


def save_file(fig, path, filename, width=1000, height=600):
    fname_png = os.path.join(path, f'{filename}.png')
    fname_svg = os.path.join(path, f'{filename}.svg')
    to_write = [fname_png]# , fname_svg]
    for f in to_write:
        fig.write_image(f, engine='kaleido', width=width, height=height)

In [118]:
# Convert Period to string for Plotly compatibility

CHART_HEIGHT=785
CHART_WIDTH=1480

# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for Acquisitions
fig.add_trace(
    go.Bar(
        x=data_fl['clean_name'],
        y=data_fl['acquisitions'],
        name="Acquisitions",
        marker_color=FIRST_COLOR
    ),
    secondary_y=False,
)

# Add trace for Dispositions (negative values)
fig.add_trace(
    go.Bar(
        x=data_fl['clean_name'],
        y=-data_fl['dispositions'],
        name="Dispositions",
        marker_color=SECOND_COLOR
    ),
    secondary_y=False,
)

# Calculate y-axis ranges
y_max = max(data_fl['acquisitions'].max(), data_fl['dispositions'].max()) * 1.15
y_min = -y_max
y_range = y_max - y_min

# Compress net effect range
net_max = max(abs(data_fl['net'].max()), abs(data_fl['net'].min()))
compression_factor = 0.6  # Adjust this value to change compression
net_range = y_range * compression_factor
net_scale = net_range / (2 * net_max)

# Scale net effect values
scaled_net = data_fl['net'] * net_scale

# Add trace for Net Effect
fig.add_trace(
    go.Scatter(
        x=data_fl['clean_name'],
        y=scaled_net,
        mode='lines+markers+text',
        name="Net Effect",
        line=dict(color='#FCC054', width=3),
        marker=dict(size=8, color='#FCC054'),
        text=data_fl['net'].round(2),
        textposition='top center',
        textfont=dict(color='white'),
        hovertemplate='%{x}<br>Net: %{text:.2f}<extra></extra>'
    ),
    secondary_y=False,
)

# Update layout for dark mode and styling
title_text = f"Institutional Single Family Home Acquisitions, Dispositions and Net Effect<br><span style='font-size: 16px; font-style: italic;'>June to August, 2024</span>".upper()

fig.update_layout(
    title={
        'text': title_text, # <br><span style='font-size: 16px; font-style: italic;'>{market_name}</span>",
        'y': TITLE_Y,
        'x': TITLE_X,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20, color='white')
    },
    font=dict(color='white', size=14),
    plot_bgcolor=BG_COLOR,
    paper_bgcolor=BG_COLOR,
    height=CHART_HEIGHT,
    width=CHART_WIDTH,
    hovermode="x unified",
    barmode='relative',
    margin=dict(l=0, r=0, t=0, b=0),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=0,
        xanchor="left",
        x=0
    )
)

# Update x-axis
fig = update_x_axis_tick_marks(fig)

# Update y-axis (Acquisitions, Dispositions, and Net Effect)
fig.update_yaxes(
    title_text="Acquisitions vs. Dispositions".upper(),
    title_font=dict(size=10),
    showgrid=True,
    gridwidth=0.5,
    gridcolor=GRID_LINES_COLOR,
    tickfont=dict(size=12),
    title_standoff=Y_AXIS_TITLE_STANDOFF,
    linecolor=CHART_BORDER_COLOR,
    linewidth=2,
    mirror=True,
    range=[y_min + (y_min*0.1), y_max + (y_max*0.1)],
    secondary_y=False
)

# Add Parcl Labs watermark
fig = lab_plot_utils(fig, logo_x=0.94)

fig.update_traces(
    textfont=dict(color='#FCC054'),
    texttemplate='<span style="text-shadow: -1px -1px 0 #000, 1px -1px 0 #000, -1px 1px 0 #000, 1px 1px 0 #000;">%{text:.0f}</span>',
    selector=dict(type='scatter', mode='lines+markers+text')
)

# Show the figure
fig.show()


# save_file(fig, path, f'{root}', width=CHART_WIDTH, height=CHART_HEIGHT)


In [152]:
['North Port, FL', 'Tampa, FL', 'Lakeland, FL', 'Jacksonville, FL']

,parcl_id,acquisitions,dispositions,net,clean_name,pct_sf_housing_stock_portfolio_1000_plus,current_portfolio_size,start_portfolio_size,port_start,port_size_change
21,2900041,131,130,1,"Lakeland, FL",2.88,6242,6204,6241,0.000160
66,2900213,394,401,-7,"Orlando, FL",2.79,18780,18579,18787,-0.000373
78,2899748,143,161,-18,"Deltona, FL",1.17,2813,2786,2831,-0.006358
81,2899989,300,322,-22,"Jacksonville, FL",3.74,17755,17645,17777,-0.001238
82,2900229,52,75,-23,"Palm Bay, FL",1.55,3050,2942,3073,-0.007485


In [155]:
data_fl['portfolio_size'] = 'PORTFOLIO_1000_PLUS'
data_out = data_fl[['parcl_id', 'clean_name', 'portfolio_size', 'acquisitions', 'dispositions', 'net', 'pct_sf_housing_stock_portfolio_1000_plus']]
data_out = data_out.rename(columns={'pct_sf_housing_stock_portfolio_1000_plus': 'pct_of_single_family_homes_owned'})
data_out['pct_of_single_family_homes_owned'] = data_out['pct_of_single_family_homes_owned']/100
data_out.to_csv('institutional_acquisitions_dispositions.csv', index=False)

In [157]:
data_out

,parcl_id,clean_name,portfolio_size,acquisitions,dispositions,net,pct_of_single_family_homes_owned
21,2900041,"Lakeland, FL",PORTFOLIO_1000_PLUS,131,130,1,0.0288
66,2900213,"Orlando, FL",PORTFOLIO_1000_PLUS,394,401,-7,0.0279
78,2899748,"Deltona, FL",PORTFOLIO_1000_PLUS,143,161,-18,0.0117
81,2899989,"Jacksonville, FL",PORTFOLIO_1000_PLUS,300,322,-22,0.0374
82,2900229,"Palm Bay, FL",PORTFOLIO_1000_PLUS,52,75,-23,0.0155
88,2900192,"North Port, FL",PORTFOLIO_1000_PLUS,102,135,-33,0.0186
92,2900417,"Tampa, FL",PORTFOLIO_1000_PLUS,564,608,-44,0.0278
93,2899822,"Cape Coral, FL",PORTFOLIO_1000_PLUS,111,155,-44,0.0188
94,2900128,"Miami, FL",PORTFOLIO_1000_PLUS,171,218,-47,0.0116


- Single family home analysis only
- Institutional Portfolios defined as 1000+ units owned in portfolio
- Tampa, Orlando and Jacksonville Stand out. In all three markets today, institutional portfolios account for nearly 1 in 20 listings over the last 60 days, with substnatial spikes occurring in September in Tampa in particular. 
- Tampa, Orlando, and Jackonsville all have a strong instiutional presence with 2.8%, 2.8%, and 3.7% of all single family homes owned in these areas by institutional portfolios. 
- In all 3 markets, institutional operators have been net sellers over the last 90 days. 

In short, Tampa, Orlando and Jacksonville are all heavily owned institutional markets, where an increasing share of all available inventory coming to market is coming from these portfolios and they have have already been reducing their exposure in these markets over the last 90 days. These are the markets to watch in my opinion as that is a lot of inventory that could come to market and these operators are typically very motivated to exit when they start selling, and will actively reduce prices to do so. 